In [30]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

# Create Torch Dataset

In [37]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        items = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{items['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(items['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [38]:
train_dataset = KeypointsDataset("data/images","data/data_train.json")
val_dataset = KeypointsDataset("data/images","data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

# Create Model

In [39]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

/opt/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [40]:
model = model.to(device)

# Train the Model

In [41]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [42]:
epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14778.0283203125
Epoch 0, iter 10, loss: 14788.2734375
Epoch 0, iter 20, loss: 14859.3671875
Epoch 0, iter 30, loss: 13871.736328125
Epoch 0, iter 40, loss: 13607.232421875
Epoch 0, iter 50, loss: 12953.1044921875
Epoch 0, iter 60, loss: 11957.548828125
Epoch 0, iter 70, loss: 11925.58984375
Epoch 0, iter 80, loss: 11598.9794921875
Epoch 0, iter 90, loss: 11606.8330078125
Epoch 0, iter 100, loss: 11797.7880859375
Epoch 0, iter 110, loss: 11422.0693359375
Epoch 0, iter 120, loss: 10032.53515625
Epoch 0, iter 130, loss: 9914.8720703125
Epoch 0, iter 140, loss: 10044.703125
Epoch 0, iter 150, loss: 9452.130859375
Epoch 0, iter 160, loss: 8961.1201171875
Epoch 0, iter 170, loss: 8687.7861328125
Epoch 0, iter 180, loss: 8082.43994140625
Epoch 0, iter 190, loss: 8498.7158203125
Epoch 0, iter 200, loss: 7763.068359375
Epoch 0, iter 210, loss: 7519.1142578125
Epoch 0, iter 220, loss: 7084.0361328125
Epoch 0, iter 230, loss: 6760.31396484375
Epoch 0, iter 240, loss: 6689.

In [43]:
torch.save(model.state_dict(), "keypoints_model.pt")